In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

data_folder = os.path.join("..", "data")
plots_folder = os.path.join("..", "plots")

# Load returns
assets = {}
for ticker in ["TSLA", "BND", "SPY"]:
    df = pd.read_csv(os.path.join(data_folder, f"{ticker}.csv"), parse_dates=[0], index_col=0)
    df["Daily_Return"] = df["Close"].pct_change()
    assets[ticker] = df

returns_df = pd.concat([assets[t]["Daily_Return"] for t in assets], axis=1)
returns_df.columns = assets.keys()
returns_df.dropna(inplace=True)

# Backtesting period
backtest = returns_df.loc["2024-08-01":"2025-07-31"]

# Example weights from Task 4
weights = {"TSLA": 0.5, "BND": 0.3, "SPY": 0.2}

# Portfolio returns
portfolio_returns = (backtest * pd.Series(weights)).sum(axis=1)
portfolio_cum = (1 + portfolio_returns).cumprod()

# Benchmark 60/40 SPY/BND
benchmark_weights = {"SPY": 0.6, "BND": 0.4}
benchmark_returns = (backtest * pd.Series(benchmark_weights)).sum(axis=1)
benchmark_cum = (1 + benchmark_returns).cumprod()

plt.figure(figsize=(12,6))
plt.plot(portfolio_cum, label="Strategy Portfolio")
plt.plot(benchmark_cum, label="60/40 Benchmark")
plt.legend()
plt.title("Backtest: Portfolio vs Benchmark")
plt.savefig(os.path.join(plots_folder, "task5_backtest.png"))
plt.show()

print(f"Final Strategy Return: {(portfolio_cum.iloc[-1] - 1)*100:.2f}%")
print(f"Final Benchmark Return: {(benchmark_cum.iloc[-1] - 1)*100:.2f}%")
